In [ ]:
import warnings
warnings.filterwarnings('ignore')

import re
import time
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import requests

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

## Question 1

Write a python program to display all the header tags from
‘en.wikipedia.org/wiki/Main_Page’.

In [ ]:
def webscraping(link,tag):
    wiki_page=requests.get(link)
    
    wiki=BeautifulSoup(wiki_page.content)
    
    titles=wiki.find_all(tag)
    
    for title in titles:
        print(title.text.replace('\n',''))

In [ ]:
wiki_link='https://en.wikipedia.org/wiki/Main_Page'
tag='h2'
tags=['h1','h2','h3']

Headings in h2 tag

In [ ]:
webscraping(wiki_link,tag)

Heading in h1,h2, and h3 tags

In [ ]:
webscraping(wiki_link,tags)

## Question 2

Write a python program to display IMDB’s Top rated 100 movies’ data (i.e. Name,
IMDB rating, Year of release) and save it in form of a CSV file. 

In [ ]:
imdb_link='https://www.imdb.com/chart/top/?ref_=nv_mv_250'
tag='td'

In [ ]:
def webscraping_imdb(link,tag):
    imdb_page=requests.get(link)
    imdb=BeautifulSoup(imdb_page.content)
    
    names=[]
    years=[]
    for x in imdb.find_all(tag,class_='titleColumn')[:100]:
        names.append(x.find('a').text)
        years.append(re.sub('[()]','',x.find('span').text))
        
    ratings=[]
    for x in imdb.find_all('td',class_='ratingColumn imdbRating')[:100]:
        ratings.append(x.text.replace('\n',''))
        
    df=pd.DataFrame({'Movie':names,'Rating':ratings,'Year_of_Release':years,})
    print(df)
    df.to_csv('imdb_top100movies.csv',index=False)

In [ ]:
webscraping_imdb(imdb_link,tag)

## Question 3

Write a python program to display IMDB’s Top rated 100 Indian movies’ data (i.e.
Name, IMDB rating, Year of release) and save it in form of a CSV file. 

In [ ]:
imdb_ind_link='https://www.imdb.com/india/top-rated-indian-movies/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=8a7876cd-2844-4017-846a-2c0876945b7b&pf_rd_r=QA4JGCD0VVTMXZ85RHJ8&pf_rd_s=right-5&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_india_tr_rhs_1'
tag='td'

In [ ]:
def webscraping_imdb_ind(link,tag):
    imdb_ind_page=requests.get(link)
    imdb_ind=BeautifulSoup(imdb_ind_page.content)
    
    names=[]
    years=[]
    for x in imdb_ind.find_all(tag,class_='titleColumn')[:100]:
        names.append(x.find('a').text)
        years.append(re.sub('[()]','',x.find('span').text))
        
    ratings=[]
    for x in imdb_ind.find_all('td',class_='ratingColumn imdbRating')[:100]:
        ratings.append(x.text.replace('\n',''))
        
    df=pd.DataFrame({'Movie':names,'Rating':ratings,'Year_of_Release':years,})
    print(df)
    df.to_csv('imdb_ind_top100movies.csv',index=False)

In [ ]:
webscraping_imdb_ind(imdb_ind_link,tag)

## Question 4

Write a python program to scrap book name, author name, genre and book review of
any 5 books from ‘www.bookpage.com’


In [ ]:
bk_link='https://bookpage.com/reviews'
tags=['h4','p','p','div']

In [ ]:
def bookpage_scraping(link,tags):
    bk_page=requests.get(link)
    bk=BeautifulSoup(bk_page.content)
    driver=webdriver.Chrome(executable_path='D:\Data Science\Web Scraping\chromedriver.exe')
    driver.get(link)
    itag=iter(tags)
    
    tag=next(itag)
    bk_titles=[]
    for x in bk.find_all(tag,class_='italic')[3:8]:
        bk_titles.append(x.text.replace('\n',''))
        
    tag=next(itag)    
    bk_authors=[]
    for x in bk.find_all(tag,class_='sans bold')[3:8]:
        bk_authors.append(x.text.replace('\n',''))
    
    tag=next(itag)
    bk_genres=[]
    for x in bk.find_all(tag,class_='genre-links hidden-phone')[3:8]:
        bk_genres.append(x.text.replace('\n',''))
        
    tag=next(itag)
    bk_reviews=[]
    for x in bk_titles:
        driver.find_element_by_link_text(x).click()
        time.sleep(0.8)
        current_bk_page=requests.get(driver.current_url)
        driver.get(link)
        time.sleep(0.8)

        current_bk=BeautifulSoup(current_bk_page.content)
        bk_reviews.append(current_bk.find(tag,class_='article-body').text.replace('\n',''))
        
    df=pd.DataFrame({'Book':bk_titles,'Author':bk_authors,'Genre':bk_genres,'Review':bk_reviews})
    return df

In [ ]:
bookpage_scraping(bk_link,tags)

## Question 5

Write a python program to scrape cricket rankings from ‘www.icc-cricket.com’. You
have to scrape:
- Top 10 ODI teams in men’s cricket along with the records for matches, points and
rating.
- Top 10 ODI Batsmen in men along with the records of their team and rating.
- Top 10 ODI bowlers along with the records of their team and rating.


In [ ]:
icc_link='https://www.icc-cricket.com/rankings/mens/team-rankings/odi'
tag='tr'

In [ ]:
def icc_scraping(link,tag):
    icc_page=requests.get(link)
    icc_odi=BeautifulSoup(icc_page.content)
    
    ranks=[]
    teams=[]
    matches=[]
    points=[]
    ratings=[]
    
    ranks.append(int(icc_odi.find(tag,class_='rankings-block__banner').find('td',class_='rankings-block__banner--pos').text))
    teams.append(icc_odi.find(tag,class_='rankings-block__banner').find('span',class_='u-hide-phablet').text)
    matches.append(int(icc_odi.find(tag,class_='rankings-block__banner').find('td',class_='rankings-block__banner--matches').text))
    points.append(icc_odi.find(tag,class_='rankings-block__banner').find('td',class_='rankings-block__banner--points').text)
    ratings.append(int(re.sub('[\n ]','',icc_odi.find(tag,class_='rankings-block__banner').find('td',class_='rankings-block__banner--rating u-text-right').text)))
    
    for team in icc_odi.find_all(tag,class_='table-body')[:9]:
        ranks.append(int(re.split('\n',team.text)[1]))
        teams.append(re.split('\n',team.text)[4])
        matches.append(int(re.split('\n',team.text)[7]))
        points.append(re.split('\n',team.text)[8])
        ratings.append(int(re.split('\n',team.text)[9]))
    
    df=pd.DataFrame({'Rank':ranks,'Team':teams,'Matches':matches,'Point':points,'Ratings':ratings})
    return df

In [ ]:
icc_scraping(icc_link,tag)

## Question 6

Write a python program to scrape cricket rankings from ‘www.icc-cricket.com’. You
have to scrape:
- Top 10 ODI teams in women’s cricket along with the records for matches, points
and rating.
- Top 10 women’s ODI players along with the records of their team and rating.
- Top 10 women’s ODI all-rounder along with the records of their team and rating.

In [ ]:
wicc_link='https://www.icc-cricket.com/rankings/womens/team-rankings/odi'
tag='tr'

In [ ]:
def wicc_scraping(link,tag):
    icc_page=requests.get(link)
    icc_odi=BeautifulSoup(icc_page.content)
    
    ranks=[]
    teams=[]
    matches=[]
    points=[]
    ratings=[]
    
    ranks.append(int(icc_odi.find(tag,class_='rankings-block__banner').find('td',class_='rankings-block__banner--pos').text))
    teams.append(icc_odi.find(tag,class_='rankings-block__banner').find('span',class_='u-hide-phablet').text)
    matches.append(int(icc_odi.find(tag,class_='rankings-block__banner').find('td',class_='rankings-block__banner--matches').text))
    points.append(icc_odi.find(tag,class_='rankings-block__banner').find('td',class_='rankings-block__banner--points').text)
    ratings.append(int(re.sub('[\n ]','',icc_odi.find(tag,class_='rankings-block__banner').find('td',class_='rankings-block__banner--rating u-text-right').text)))
    
    for team in icc_odi.find_all(tag,class_='table-body')[:9]:
        ranks.append(int(re.split('\n',team.text)[1]))
        teams.append(re.split('\n',team.text)[4])
        matches.append(int(re.split('\n',team.text)[7]))
        points.append(re.split('\n',team.text)[8])
        ratings.append(int(re.split('\n',team.text)[9]))
    
    df=pd.DataFrame({'Rank':ranks,'Team':teams,'Matches':matches,'Point':points,'Ratings':ratings})
    return df

In [ ]:
wicc_scraping(wicc_link,tag)

## Question 7

Write a python program to scrape details of all the mobile phones under Rs. 20,000
listed on Amazon.in. The scraped data should include Product Name, Price, Image URL
and Average Rating.

In [ ]:
amazon_page=requests.get('https://www.amazon.in/s?bbn=1389401031&rh=n%3A1389401031%2Cp_36%3A1318506031&dc&qid=1620500039&rnid=1318502031&ref=lp_1389401031_nr_p_36_3')

In [ ]:
amazon_page

- Hence Response is not 200 we cannot scrap this page data

Scraping Flipkart data Instead

In [ ]:
flip_link='https://www.flipkart.com/search?sid=tyy%2C4io&otracker=CLP_Filters&p%5B%5D=facets.price_range.from%3DMin&p%5B%5D=facets.price_range.to%3D20000&p%5B%5D=facets.rating%255B%255D%3D4%25E2%2598%2585%2B%2526%2Babove'

In [ ]:
def flipkart_scraping(links):
    flipkart_page=requests.get(links)
    flipkart=BeautifulSoup(flipkart_page.content)
    driver=webdriver.Chrome(executable_path='D:\Data Science\Web Scraping\chromedriver.exe')
    driver.get(links)
    
    mobiles=[]
    for x in flipkart.find_all('div',class_='_2kHMtA'):
        mobiles.append(x.find('div',class_='_4rR01T').text)
        
    prices=[]
    for x in flipkart.find_all('div',class_='_2kHMtA'):
        prices.append(x.find('div',class_='_30jeq3 _1_WHN1').text)
        
    urls=[]    
    for link in driver.find_elements_by_xpath('//div[@class="_2kHMtA"]/a'):
        driver.get(link.get_attribute('href'))
        time.sleep(.8)
        urls.append(driver.find_element_by_xpath('//div[@class="CXW8mj _3nMexc"]/img').get_attribute('src'))

        driver.get(links)
        time.sleep(.8)
        
    df=pd.DataFrame({'Mobile':mobiles,'Price':prices,'Image_URL':urls})
    return df

In [ ]:
flipkart_scraping(flip_link)

## Question 8

Write a python program to extract information about the local weather from the National
Weather Service website of USA, https://www.weather.gov/ for the city, San
Francisco. You need to extract data about 7 day extended forecast display for the city.
The data should include period, short description, temperature and description. 

In [ ]:
weather_link='https://forecast.weather.gov/MapClick.php?lat=37.777120000000025&lon=-122.41963999999996#.YJep3bUzZPY'

In [ ]:
def weather_scraping(link):
    wheather_page=requests.get(link)
    wheather=BeautifulSoup(wheather_page.content)
    
    periods=[]
    temps=[]
    
    for x in wheather.find_all('li',class_='forecast-tombstone'):
        periods.append(x.find('p').text)
        temps.append(x.find('p',class_='temp temp-low').text)
        
    decps=[]
    
    for x in wheather.find_all('div',class_='row row-odd row-forecast'):
        decps.append(x.find('div',class_='col-sm-10 forecast-text').text)
        
    df=pd.DataFrame({'Period':periods,'Temperature':temps,'Description':decps})
    return df

In [ ]:
weather_scraping(weather_link)